<a href="https://colab.research.google.com/github/jfink09/Algorithmic-Trading/blob/main/WritingAlogrithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

### Set Dates

In [ ]:
self.SetStartDate(2018,1,1)
self.SetEndDate(2023,1,1)

# Set the end date to last week
self.SetEndDate(datetime.now() - timedelta(7))

### Set Account Currency

In [ ]:
# Defaults to USD
self.SetAccountCurrency("USD")

### Set Cash

In [ ]:
# Live trading defaults to account value
self.SetCash(10000)

In [ ]:
### Set Brokerage and Cash Model
self.SetBrokerageModel(BrokerageName.TDAmeritrade,AccountType.Margin)

### Universe Settings

In [ ]:
self.UniverseSettings.Resolution = Resoultion.Minute
self.AddUniverse(self.CoarseFilterFunction)

### Set Security Initializer

In [ ]:
# In initialize
self.SetSecurityInitializer(self.CustomSecurityInitializer)

def CustomSecurityInitializer(self, security: Security) -> None:
  # Disable trading fees
  security.SetFeeModel(ConstantFeeModel(0,"USD"))

self.SetSecurityInitializer(lambda security: security.SetFeeModel(ConstantFeeModel(0,"USD")))

In [ ]:
# Use a security initializer to set the market price of each security to the last known price
seeder = FuncSecuritySeeder(self.GetLastKnownPrices)
self.SetSecurityInitializer(lambda security: seeder.SeedSecurity(security))

In [ ]:
# In initialize
self.SetSecurityInitializer(MySecurityInitializer(self.BrokerageModel, FuncSecuritySeeder(self.GetLastKnownPrices)))

# Outside of the algorithm class
class MySecurityInitializer(BrokerageModelSecurityInitializer):
  def __init__(self,brokerage_model: TDAmeritrade, security_seeder: ISecuritySeeder) -> None:
    super().__init__(brokerage_model, security_seeder)
  def Initialize(self, security: Security) -> None:
    # First call the superclass definition
    # This method sets the reality models of each security using the default reality models of the brokerage model
    super().Initialize(security)

    # Next, overwrite some of the reality models
    security.SetFeeModel(ConstantFeeModel(0,"USD"))

### Add Data

In [ ]:
# Adds minut bars by default
self.AddEquity("SPY")

# Use second bars instead
self.AddEquity("SPY", Resolution.Second)

In [ ]:
# Set Indicators and Consolidators
self.symbol = self.AddEquity("SPY").Symbol
self.sma = self.SMA(self.symbol, 21)
self.WarmUpIndicator(self.symbol, self.sma)

### Set Algorithm Settings

In [ ]:
self.Settings.RebalancePortfolioOnSecurityChanges = False

### Set The Time Zone

In [ ]:
self.SetTimeZone("NewYork")
self.SetTimeZone(TimeZones.NewYork)

### Set Warmup Period

In [ ]:
self.SetWarmUp(timedelta(7))

# Feed in 100 trading days worth of data before the start date
self.SetWarmUp(100, Resolution.Daily)

# If you do not provide the Resolution argument, it uses the lowest resoultion in the subscriptions
self.SetWarmUp(100)

### Post Initalizations

In [ ]:
def OnWarmUpFinished(self) -> None:
  self.Log("Algorithm Ready")